# Project 2: Corona/Weather Data Science - FYP 2021
## How weather influences the spread of the pandemic 
---
### Group 9: Aidan Stocks, Christian Margo Hansen, Jonas-Mika Senghaas, Malthe Pabst, Rasmus Bondo Hansen
Submission: 19.03.2021 / Last Modified: 18.03.2021

---

This notebook contains the step-by-step data science process performed on the `IBM Weather Data` from 2020 and the official `Corona Statistics` in 2020. 
The goal was to inform the authorities of **Germany** (*Group 9 Focus*) about the development of the pandemic in 2020 investigate possible relations between environmental conditions and the spread of the pandemic.

The raw datasets were given by the course mananger. ([Corona Germanu](https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0/data?orderBy=Meldedatum), [Weather IBM]())

> *Contact for technical difficulties or questions related to code: Jonas-Mika Senghaas (jsen@itu.dk), Rasmus Bondo (rabh@itu.dk)*

### Introduction
The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). It was first identified in December 2019 in Wuhan, China. The World Health Organization declared the outbreak a Public Health Emergency of International Concern in January 2020 and a pandemic in March 2020. As of 5 March 2021, more than 115 million cases have been confirmed, with more than 2.56 million deaths attributed to COVID-19, making it one of the deadliest pandemics in history. (Source: [Wikipedia](https://en.wikipedia.org/wiki/COVID-19_pandemic)).

The goal of this project is to analyse the development of the pandemic in Germany from February 2020, while paying special attention to environmental factors that may relate to higher chances of infection and therefore a faster spread of the pandemic.

## Running this Notebook
---
This notebook contains all code to reproduce the findings of the project as can be seen on the [GitHub](https://github.com/jonas-mika/fyp2021p02g09) page of this project. In order to read in the data correctly, the global paths configured in the section `Constants` need to be correct. The following file structure - as prepared in the `submission.zip` - was followed throughout the project and is recommended to use (alternatively the paths in the section `Constants` can be adjusted):

```
submission
|   github_link.webloc
│   project_report.pdf
│   gitlog.txt    
│
└───data
│   │
│   | raw
│   | │   corona
│   | │   metadata
│   | │   shapefiles
|   | |   weather
│   
└───notebooks
    |   project2.ipynb (current location)
    |   project2
        |   __init__.py
        |   visualisations.py
        |   ...
    
```
*Note that the rest of the file structure as can be seen on the [GitHub](https://github.com/jonas-mika/fyp2021p02g09) page of the project generates automatically (ie. `interim` data/ `processed` data/ all `reports`)*

# Required Libraries
---
Throughout the project, we will use a range of both built-in and external Python Libraries. This notebook will only run if all libraries and modules are correctly installed on your local machines. 
To install missing packages use `pip install <package_name>` (PIP (Python Package Index) is the central package management system, read more [here](https://pypi.org/project/pip/)). 

In case you desire further information about the used packages, click the following links to find detailled documentations:
- [Pandas Homepage](https://pandas.pydata.org/)
- [Numpy Homepage](https://numpy.org/)
- [Matplotlib Homepage](https://matplotlib.org/stable/index.html)
- [Folium Documentation](https://python-visualization.github.io/folium/)
- [Scipy Homepage](https://www.scipy.org/)

In [ ]:
import pandas as pd                                    # provides major datastructure pd.DataFrame() to store the datasets
import geopandas as gpd                                # used for loading geojson data for spatial visualisation
import numpy as np                                     # used for numerical calculations and fast array manipulations
import matplotlib.pyplot as plt                        # visualisation of data
import matplotlib.dates as mdates                      # provides functionality for intelligent axis labelling with datetime objects
import statsmodels.api as sm                           # 
from scipy.stats import pearsonr, spearmanr            # runs `pearson` and `spearman` association test of numerical variables on two variables
import datetime                                        # converts strings to datetime objects
import seaborn as sns                                  
import folium                                          # spatial visualisation
import json                                            # data transfer to json format
import os                                              # automates saving of export files (figures, summaries, ...)
import random                                          # randomness in coloring of plots
import re                                              # used for checking dateformat in data cleaning

## Own Package
---
Since this project makes heavy use of functions to achieve maximal efficiency, all functions are stored externally in the package structure `project1'. The following imports are necessary for this notebook to run properly.

In [ ]:
from project2.processing import check_columns_for_missing_values
from project2.save import save_csv, save_json, save_figure, save_dict, save_map
from project2.visualisations import categorical_scatterplot

**REMARK**: All function used in this project are well documented in their `Docstring`. To display the docstring and get an short summary of the function and the specifications of the input argument (including data tupe and small explanation) as well as their return value, type **`?<function_name>`** in Juptyer (*see example*)

# Constants
---
To enhance the readibilty, as well as to decrease the maintenance effort, it is useful for bigger projects to define contants that need to be accessed globally throughout the whole notebook in advance. 
The following cell contains all of those global constants. By convention, we write them in caps (https://www.python.org/dev/peps/pep-0008/#constants)

In [ ]:
# path lookup dictionary to store the relative paths from the directory containing the jupyter notebooks to important directories in the project
PATH = {}

PATH['data'] = {}
PATH['data']['raw'] = "../data/raw/"
PATH['data']['interim'] = "../data/interim/"
PATH['data']['processed'] = "../data/processed/"
PATH['data']['external'] = "../data/external/"

PATH['corona'] = 'corona/'
PATH['metadata'] = 'metadata/'
PATH['shapefiles'] = 'shapefiles/'
PATH['weather'] = 'weather/'

PATH['reports'] = "../reports/"

# filename lookup dictionary storing the most relevant filenames
FILENAME = {}
FILENAME['weather'] = 'weather.csv'
FILENAME['corona'] = 'de_corona.csv'
FILENAME['metadata'] = 'de_metadata.json'
FILENAME['shapefiles'] = 'de.geojson'

# defining three dictionaries to store data. each dictionary will reference several pandas dataframes
DATA_RAW = {}
DATA_GERMANY = {}
DATA_EXTERNAL = {}

REGION_LOOKUP = {}

# automising all plots requires a lot of additional information (ie. what plot-type to use on the different variables, whether or not we need a fivenumber-summary, etc.). this information is stored in the summary dictionary
SUMMARY = {}
    
NAMES = {}
NAMES['datasets'] = ['weather', 'corona']
NAMES['jsons'] = ['metadata', 'shapefiles']

In [ ]:
# load in metadata using json library
for JSON in NAMES['jsons']:
    with open(PATH['data']['raw'] + PATH[JSON] + FILENAME[JSON]) as infile:
        DATA_GERMANY[JSON] = json.load(infile)

In [ ]:
REGION_LOOKUP['iso'] = {
    DATA_GERMANY['metadata']['country_metadata'][i]['iso3166-2_code']: 
    DATA_GERMANY['metadata']['country_metadata'][i]['iso3166-2_name_en'] 
        for i in range(len(DATA_GERMANY['metadata']['country_metadata']))} # key: iso, value: region name

REGION_LOOKUP['region'] = {
    DATA_GERMANY['metadata']['country_metadata'][i]['iso3166-2_name_en']: 
    DATA_GERMANY['metadata']['country_metadata'][i]['iso3166-2_code']                            
        for i in range(len(DATA_GERMANY['metadata']['country_metadata']))} # key: region name, value: iso

# uncomment if you experience problems with germany special character 'ü'
# REGION_LOOKUP['iso']['DE-BW'] = 'Baden-Wuerttemberg'
# REGION_LOOKUP['iso']['DE-TH'] = 'Thueringen'
# REGION_LOOKUP['region']['Thueringen'] = 'DE-TH'
# REGION_LOOKUP['region']['Baden-Wuerttemberg'] = 'DE-BW'
# del REGION_LOOKUP['region']['Thüringen']
# del REGION_LOOKUP['region']['Baden-Württemberg']

# Loading, Inspection and Processing of Datasets (TASK 0)
---

## Loading in the Datasets
---
The data analysis revolves around a handful of datasets from different resources: 
> *CSV*: Corona (DE) -  Contains the `Number of new infections (per day)` and `Number of new casualties (per day)` filtered by day and region in Germany.

> *CSV*: Weather - Contains information about several indicators of weather conditions for each region in Germany, Denmark, Sweden and the Netherlands for each day during the period 13.02.2020 - 21.02.2021 (if `weather.csv` and `weather2.csv` are combined)

> *JSON*: Metadata (DE) - Contains more information about the different regions in Germany

> *GEOJSON*: Geojson (DE) - Holds the `geojson` data for the different regions in Germany

We conveniently load in the `csv` datasets into individual Pandas `DataFrames` using the built-in pandas method `pd.read_csv()` ([Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)). We store those in our `DATA_RAW` dictionary in the corresponding keys.

For the `json` and `geojson` files, we use Pythons built-in library `json` in order to store their content in Python `dicts`.

In [ ]:
# load in weather and corona data using pandas into the predefined dictionary 'DATA_RAW'
for dataset in NAMES['datasets']:
    DATA_RAW[dataset] = pd.read_csv(PATH['data']['raw'] + PATH[dataset] + FILENAME[dataset], sep = '\t')

Unfortunately, the weather data was given in two separate dataframes, where `weather.csv` contains data in the period of 13.02.2020 - 14.11.2020 and `weather2.csv` contains data in the period 15.11.2020 - 21.02.2021.

For our analysis we want to consider all weather records in the combined time periods. We therefore read in the `weather2.csv` into another `pd.DataFrame` and then use `pd.concat` to combine our intially loaded weather data with the additional weather data along the vertical axis (`axis=0`).

In [ ]:
additional_weather = pd.read_csv(PATH['data']['raw'] + PATH['weather'] + 'weather2.csv', sep= '\t')
DATA_RAW['weather'] = pd.concat([DATA_RAW['weather'], additional_weather])

## Inspection of Datasets
---
We can now have a look at our two dataset to get a good first impression for what kind of data we are dealing with. We start by reporting the number of records and fields/  variables in each of the datasets by using the `shape` property of the Pandas `DataFrame`. 
We then continue to have an actual look into the data. Similiar to the `head` command in terminal, we can use the method `head()` onto our `DataFrames`, which outputs a nice inline representation of the first five data records of the dataset.

### Time Period

In [ ]:
for dataset in NAMES['datasets']:
    print(f"{dataset.capitalize()}: {DATA_RAW[dataset]['date'].iloc[0].replace('-','.')} - {DATA_RAW[dataset]['date'].iloc[-1].replace('-','.')}")

### Size (Number of Records and Fields)

In [ ]:
for dataset in NAMES['datasets']:
    print(f"{dataset.capitalize()}: {DATA_RAW[dataset].shape}")

### Peek into Datasets (Describing Attributes)

In [ ]:
DATA_RAW['weather'].head()

We see, that the main dataset `weather` stores weather information for the four countries Germany (DE), Denmark (DK), Sweden (SE) and the Netherlands (NL) . It consist of 16.953 rows, corresponding to reports of weather conditions on a specified day in a specified region in one of the countries. The dataset holds the weather data recorded for each of the region from the 13th of February 2020 to the 21st of February 2021. The following variables appear in the dataset:

> `DATE` (YYYY-MM-DD): The day of the weather reports (*Time Attribute*)

> `ISO 3166-2`: [ISO 2 Code]() for the region, in which the weather report is (*Geographic Attribute*)

> `RELATIVE HUMIDITY SURFACE` (%): The **daily averaged** ratio of the partial pressure of water vapour to the equilibrium vapour pressure of water at the same temperature near the surface.(*Numerical Attribute*)

> `SOLAR RADIATION` (W/m^2): The **daily sum** of the electromagnetic radiation given off by the sun (*Numerical Attribute*)

> `SURFACE PRESSURE` (Pa): The **daily sum** pressure of the atmosphere at the earth's surface (*Numerical Attribute*)

> `TEMPERATURE ABOVE GROUND` (Kelvin): The **daily averaged** temperature of the air some meters above (*Numerical Attribute*)

> `TOTAL PRECIPITATION` (mm/s): The **daily sum** precipitation is an averaged precipitation rate over 3 hours (*Numerical Attribute*)

> `UV INDEX` ([Computation](https://en.wikipedia.org/wiki/Ultraviolet_index)): The **daily sum** of the strength of sunburn-producing ultraviolet radiation (*Numerical Attribute*)

> `WIND SPEED` (m/s): The **daily averaged** measured wind speed (*Numerical Attribute*)

In [ ]:
DATA_RAW['corona'].head()

We see, that the dataset `corona` holds information about newly infected cases and new deaths per day per region in Germany for most of the days in 2020. It consist of 5602 rows, corresponding to 5.602 reports of new infections and deaths per day in the different regions of Germany. 

> `DATE` (YYYY-MM-DD): The day of the reports of new infections and deaths per region

> `REGION CODE`: Region in Germany (*Geographic Attribute*)

> `CONFIRMED ADDITION`: The number of newly confirmed infections in the region on the specified day (*Numerical Attributes*)

> `DECEASED ADDITION`: The number of newly confirmed deaths in the region on the specified day (*Numerical Attributes*)

## Initial Sanity Check
---
Before continuing with the data analysis, we want to make sure that the datasets are clean. There are a plentiful of methods to check this. In the following, we will stick to the following three:
>(a) **Date Representation** (*Are the dates entered in consistent syntax?*)

>(b) **Missing Values in Columns** (*Are there missing values? If yes, how many and in which columns?*)

>(c) **Missing Values in Rows** (*Are there days, where there haven't been reports of the weather or corona development?*)

### Date Representation

In [ ]:
def check_date_representation():
    for date in DATA_RAW[dataset]['date']:
        if not re.match('\d\d\d\d-\d\d-\d\d', date):
            return False
    return True

In [ ]:
for dataset in NAMES['datasets']:
    print(f"{dataset.capitalize()}: {check_date_representation()}")

### Missing Values
We check in all columns in all datasets the number of missing values (encoded as `' '` (*empty strings*)) to see if there are missing values (empty string) to get a feeling on which columns we need to further do processing.

In [ ]:
for dataset in NAMES['datasets']:
    print(f"{dataset.capitalize()}: {check_columns_for_missing_values(DATA_RAW[dataset])}")

## Process Data
---
As our sanity checks have proven, the provided data is already quite clean. In all datasets, there are neither values missing nor inconcistencies in the recording of data (ie. representation of dates). 

However, to make our futher analysis more pleasant, there are some minor changes that we will perform on our datasets:
> (a) **Renaming of Columns** (The naming of the columns in the original dataframes is inconcistent and - in parts - poorly descriptive (What are ie. `Confirmed Addition`?))

> (b) **Relative Data** (In our later analysis, we do not only want to look at absolute values, but want to interpret those in relation to the size of the region we are looking at)

> (c) **Metric Change** (Here we change the `Temperature` that was recorded in Kelvin into Celcius for easier readability)

### a. Renaming of Columns

In [ ]:
DATA_RAW['weather'].rename(columns={
    'date': 'date', 
    'iso3166-2': 'region_code',
    'RelativeHumiditySurface': 'relative_humidity_surface', 
    'SolarRadiation': 'solar_radiation', 
    'Surfacepressure': 'surface_pressure', 
    'TemperatureAboveGround': 'temperature_above_ground', 
    'Totalprecipitation': 'total_precipiation', 
    'UVIndex': 'uv_index', 
    'WindSpeed': 'wind_speed'}, inplace=True)

In [ ]:
DATA_RAW['corona'].rename(columns={
    'date': 'date', 
    'region_code': 'region', 
    'confirmed_addition': 'absolute_infections', 
    'deceased_addition': 'absolute_deaths'}, inplace=True)

### b. Region Codes/ Names

In [ ]:
DATA_RAW['corona']['region_code'] = DATA_RAW['corona']['region'].map(REGION_LOOKUP['region'])
DATA_RAW['weather']['region'] = DATA_RAW['weather']['region_code'].map(REGION_LOOKUP['iso'])

### c. Relative Data

In [ ]:
population_map = {DATA_GERMANY['metadata']['country_metadata'][i]['iso3166-2_name_en']:
                  DATA_GERMANY['metadata']['country_metadata'][i]['population'] 
                  for i in range(len(DATA_GERMANY['metadata']["country_metadata"]))}
                  
DATA_RAW['corona']['population'] = DATA_RAW['corona']['region'].map(population_map)
DATA_RAW['corona']['infections_(per_100.000)'] = DATA_RAW['corona']['absolute_infections'] / DATA_RAW['corona']['population'] * 100_000
DATA_RAW['corona']['deaths_(per_100.000)'] = DATA_RAW['corona']['absolute_deaths'] / DATA_RAW['corona']['population'] * 100_000

 ### d. Metric Change

In [ ]:
DATA_RAW['weather']['temperature_above_ground'] = DATA_RAW['weather']['temperature_above_ground'] - 273.15

In [ ]:
#for region in DATA_GERMANY['corona'].keys():
#    DATA_GERMANY['corona'][region]['date'] = pd.to_datetime(DATA_GERMANY['corona'][region]['date'], format='%Y-%m-%d')

### Saving Pre-Processed Data
---
We can now saave our pre-procssed data to the `../data/interim/`, which saves our data into `csv` format for later inspection.

In [ ]:
for dataset in NAMES['datasets']:
    save_csv(DATA_RAW[dataset], path=PATH['data']['interim'] + PATH[dataset], filename=f"{dataset}", index=False) 

## Filtering Datasets 
---
We got a good first feeling for both datasets, have proven them to be clean and made some adjustments to the way the data is represented, in order to make the futher analysis process easier. The next step is to filter both datasets to only hold data related to Germany. Our goal is to not only have a dataframe that holds the `weather` and `corona` data for the whole of Germany, but also for each of the 16 regions in Germany.

### Entire Germany
We therefore start by defining another empty dictionary in `DATA_GERMANY['corona']` and `DATA_GERMANY['weather']`. These will hold the `corona` and `weather` data for the whole of Germany at key `all` and each region's at the corresponding `region_name` as a key.

In [ ]:
for dataset in NAMES['datasets']:
    DATA_GERMANY[dataset] = {}

The `corona` and `weather` data for entire Germany is easy to obtain. Since the `corona` dataset is already filtered for Germany, we only need to copy the raw corona data into the key `all`.
The `weather` data, however, contains information for four countries, namely Denmark, Sweden, Netherlands and Germany. Here we need to filter. We do this by the unique region_codes that are used to map the weather data to the different secondary-class regions of each country. We can obtain these codes from the metadata provided for Germany that is saved at `DATA_RAW['metadata']`

In [ ]:
region_codes = list(REGION_LOOKUP['iso'].keys())

DATA_GERMANY['corona']['all'] = DATA_RAW['corona']
DATA_GERMANY['weather']['all'] = DATA_RAW['weather'][DATA_RAW['weather']['region_code'].isin(region_codes)]

### Regional Filtering
We now aim to filter the obtained datasets that hold the `corona` and `weather` data for all of Germany for each of the 16 regions and save them at the corresponding key in `DATA_GERMANY`. We do this by iterating over the region codes (ISO 3166-2 Standard) that we obtained from the metadata. 

In the `weather` dataset, we can use these iso-values to use a simple mask onto the german weather data in order to filter for each region. 
In the `corona` dataset, however, the regions are not defined by their iso-value, but their regular name. In order to filter correctly, we use our `REGION_LOOKUP` mapping that for an iso-value as key, which returns the regular region name. 

In [ ]:
for dataset in NAMES['datasets']:
    for region in region_codes:
        mask = DATA_GERMANY[dataset]['all']['region_code'] == region
        DATA_GERMANY[dataset][REGION_LOOKUP['iso'][region].lower()] = DATA_GERMANY[dataset]['all'][mask].reset_index()

### Inspection
---
With our newly filtered datasets, we have reduced the number of `weather` and `corona` data both to the whole of Germany and to each of its regions, and we can convienently access them by looking them up in `DATA_GERMANY[dataset]` using the region name as a key. To see, with what kind of data we are dealing now, it makes to recompute the size of each of the datasets. However, our analysis of the column attributes still holds, since we have only filtered across `axis=0` (the rows), while we left the attributes untouched. 

Again, we report the number of records and fields/ variables in each of the datasets by using the `shape` property of the Pandas `DataFrame`. 

In [ ]:
for dataset in NAMES['datasets']:
    print(f"{'-'*5}{dataset.capitalize()}{'-'*5}")
    for key in DATA_GERMANY[dataset].keys():
        print(f"{key.title()}: {DATA_GERMANY[dataset][key].shape}")

### Saving Filtered Data
---
We can now saave our filtered data to the `../data/processed/`, which saves our filtered data into `csv` format for later inspection.

In [ ]:
for dataset in NAMES['datasets']:
    for key in DATA_GERMANY[dataset].keys():
        save_csv(DATA_GERMANY[dataset][key], path=PATH['data']['processed'] + PATH[dataset], filename=f"{dataset}_{key}", index=False) 

# Single Variable Analysis (TASK 1)
---
We have obtained filtered and processed datasets from TASK 0. We will now turn our focus to analysing the attributes in each of the datasets both in a numerical summary (ie. fivenumber summaries) and visual representations. 

## Numerical Analysis
---


In [ ]:
for dataset in NAMES['datasets']:
    for region in DATA_GERMANY['weather'].keys():
        if region == 'all': 
            numerical_summary = DATA_GERMANY[dataset][region].describe().reset_index().rename(columns={'index': 'metric'})
        else: 
            numerical_summary = DATA_GERMANY[dataset][region].describe().reset_index().rename(columns={'level_0': 'metric'}).drop(['index'], axis=1)
        
        save_csv(numerical_summary, path=PATH['reports'] + f"single_variable_analysis/{region}/", filename=f'{dataset}_numerical_summary', index=False)

## Visualisation
---
An essential step of each data exploration is to visualise our data. Only this makes the abstract numbers and characters meaningful.

In the case of our two datasets holding data about daily `corona` infections and deaths and `weather` conditions, we want to plot the development in these variables over time (Feb 2020-Feb 2021). 
We want to investigate this development not only for the whole of Germany, but also for the individual regions, as well as for different levels of aggregation.

Following this logic, we will get plots for each `weather` attribute as well as `infection` and `deaths` for each region individually and the whole of Germany, and in three different levels of aggregation, `daily`, `weekly` and `monthly`.

In [ ]:
def plot_weather(data, region, condition, aggregation='daily', dimensions=(16,9)):
    fig = plt.figure(figsize=dimensions)
    ax = fig.add_axes([.15,.15,.7,.7]) # [left, bottom, width, height]

    region_data = data[region]
    region_data['date'] = pd.to_datetime(region_data['date'], format='%Y-%m-%d')

    # data to plot for each level of aggregation
    if region == 'all':
        region_data = pd.DataFrame(region_data.groupby('date')[condition].mean()).reset_index()

    if aggregation == 'daily':
        to_plot = region_data[['date', condition]]
    elif aggregation == 'weekly':
        to_plot = pd.DataFrame(region_data.groupby(pd.Grouper(key='date', freq='W-MON'))[condition].mean()).reset_index()
    elif aggregation == 'monthly':
        to_plot = pd.DataFrame(region_data.groupby(pd.Grouper(key='date', freq='M'))[condition].mean()).reset_index()

    if region == 'all': title = f"{condition.title().replace('_',' ')} in Germany"        
    else: title = f"{condition.title().replace('_',' ')} in {region.title()}"

    # plot
    ax.plot(to_plot['date'], to_plot[condition], '-o', color='darkblue', linewidth='2')

    # labeling
    ax.set_title(title, fontweight='bold', fontsize=14)
    ax.set_xlabel("Date")
    ax.set_ylabel(f"{condition.title().replace('_', ' ')}")

    # labelling of x-axis (https://stackoverflow.com/questions/9627686/plotting-dates-on-the-x-axis-with-pythons-matplotlib)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gcf().autofmt_xdate()

    ax.set_ylim(min(data['all'][condition]), max(data['all'][condition]))

    ax.text(.76,.89, f"{aggregation.capitalize()} Records", verticalalignment='center', transform = ax.transAxes, fontweight='bold')
    ax.text(.76,.84, f"Period: Feb 2020-Feb 2021", verticalalignment='center', transform = ax.transAxes)

    # activate grid
    ax.grid(True)

    return fig

In [ ]:
def plot_covid_cases(data, region, focus='infections', scale='absolute', aggregation='daily', dimensions=(16,9)):
    fig = plt.figure(figsize=dimensions)
    ax = fig.add_axes([.15,.15,.7,.7]) # [left, bottom, width, height]

    condition = f"{scale}_{focus}"

    region_data = data[region]
    region_data['date'] = pd.to_datetime(region_data['date'], format='%Y-%m-%d')

    # data to plot for each level of aggregation
    if region == 'all':
        region_data = pd.DataFrame(region_data.groupby('date')[condition].sum()).reset_index()

    if aggregation == 'daily':
        to_plot = region_data[['date', condition]]
    elif aggregation == 'weekly':
        to_plot = pd.DataFrame(region_data.groupby(pd.Grouper(key='date', freq='W-MON'))[condition].sum()).reset_index()
    elif aggregation == 'monthly':
        to_plot = pd.DataFrame(region_data.groupby(pd.Grouper(key='date', freq='M'))[condition].sum()).reset_index()

    if region == 'all': title = f"{scale.title()} Number of {focus.title()} in Germany"        
    else: 
        title = f"{scale.title()} Number of {focus.title()} in {region.title()}"

    if region != 'all' and aggregation == 'daily':
        ax.set_ylim(0, max(data['all'][condition]))
                    
    # plot
    ax.plot(to_plot['date'], to_plot[condition], '-o',color='darkblue', linewidth=2)

    # labeling
    ax.set_title(title, fontweight='bold', fontsize=14)
    ax.set_xlabel("Date")
    ax.set_ylabel(f"{scale.capitalize()} Number of {focus.capitalize()}")

    # labelling of x-axis (https://stackoverflow.com/questions/9627686/plotting-dates-on-the-x-axis-with-pythons-matplotlib)
    #plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    #plt.gca().xaxis.set_major_locator(mdates.DayLocator())
    #plt.gcf().autofmt_xdate()
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=4))

    ax.text(.13,.90, f"{aggregation.capitalize()} Records", verticalalignment='center', transform = ax.transAxes, fontweight='bold')
    ax.text(.13,.86, f"Period: Feb 2020-Feb 2021", verticalalignment='center', transform = ax.transAxes)

    # activate grid
    ax.grid(True)

    return fig

## Saving Figures
---
...

**REMARK**: Be aware that the cell takes some time to compute, since it plots and saves all figures at once.

In [ ]:
%%capture 
# all weather plots
for region in DATA_GERMANY['weather'].keys():
    for condition in list(DATA_GERMANY['weather']['all'])[2:]:
        for aggregation in ['daily', 'weekly', 'monthly']:
            try: save_figure(figure = plot_weather(data = DATA_GERMANY['weather'], region = region, condition = condition, aggregation = aggregation),
                    path = PATH['reports'] + f"single_variable_analysis/{region}/weather/{aggregation}", filename = f"{condition.lower().replace(' ', '_')}", save_to = 'pdf')
            except: None

In [ ]:
%%capture 
# all covid plots
for region in DATA_GERMANY['weather'].keys():
    for condition in ['infections', 'deaths']:
        for scale in ['absolute', 'relative']:
            for aggregation in ['daily', 'weekly', 'monthly']:
                save_figure(
                    figure = plot_covid_cases(
                        data = DATA_GERMANY['corona'], 
                        region=region, 
                        focus=condition, 
                        scale=scale, 
                        aggregation=aggregation),                                
                    path = PATH['reports'] + f"single_variable_analysis/{region}/{condition}/{scale}", filename = f"{scale}_{condition}_{aggregation}", save_to = 'pdf')

## Reports of Single Variable Analysis
---
`TASK 1` specifically asks us to report the key statistics and plots for the variables relevant for our further data analysis. Since we are interested into which environmental conditions might be affecting the spread of the virus, the first key variable is the number of new infections per day. 

In our further analysis we will analyse in detail whether there exists a correlation between the `infections` and `temperature`. Therefore we will plot these at this point.f

The following reports can also be found `../reports/`.

### Infections

In [ ]:
for region in ['all', 'hamburg']:
    for aggregation in ['daily', 'weekly', 'monthly']:
            plot_covid_cases(
                data = DATA_GERMANY['corona'],
                region = region,
                focus = 'infections',
                scale = 'absolute',
                aggregation = aggregation
            )

### Temperature

In [ ]:
for region in ['all', 'hamburg']:
    for aggregation in ['daily', 'weekly', 'monthly']:
        plot_weather(
            data = DATA_GERMANY['weather'],
            condition = 'uv_index',
            region = region,
            aggregation = aggregation
        )

# Associations (TASK 2)
---
An equally important, and perhaps more challenging, part of any data analysis is to find associations in the datasets. Finding associations for the scope of this project may influence political decision-making, since possible correlations may lead to new strategies in combating the spread of the virus. 

The following sections explain the step-by-step process of analysing the `weather` and `corona` data for significant correlations.

## Analysis Tools
---
This section gives a brief introduction into the statistical background of the analysing of correlation.

If we say `correlation`, we mean that for two observed random variables a change in one, determines a change in the other. So given some random variale `A`, we are able to (roughly) predict the observed `B`. 

**Covariance and Correlation Coefficients**

Correlations are usually measured through a so-called `correlation coefficient`, which is a statistical measure to analyse the extent to which two variables tend to change together. The coefficient describes both the strength and the direction of the relationship. The following conclusion can be drawn:

**Direction**
A relationship can either be *negativ* (-1 < `coef` < 0), if an increase in some observed variable `A` prompts a decrease in `B`. Similarly, a relationship is said to be positive (0 < `coef` < 1), if an incresaee in some `A` prompts an increase in `A`.

**Strength**
A correlation coefficient of 0 implies no correlation between the two variables at all. From the interval from 0 to both -1 and 1 the strength of the correlation increases, reaching the (unrealistic) value of -1 and 1 itself, which model a perfect correlation between the two variables.

**Statistical Testing**

For our project, we used two different statistical tests.

1. **Pearson Product Moment Correlation**

The Pearson correlation evaluates the linear relationship between two continuous variables. A relationship is linear when a change in one variable is associated with a proportional change in the other variable.

2. **Spearman Rank Order Correlation**

The Spearman correlation evaluates the monotonic relationship between two continuous or ordinal variables. In a monotonic relationship, the variables tend to change together, but not necessarily at a constant rate. The Spearman correlation coefficient is based on the ranked values for each variable rather than the raw data, and performs the Pearson calculation on the 'ranked' data.

*Note that all presented methods only test for linear relationships between the two observed variables. Relationships of some other kind are not captured within this association analysis.*

**Statistical Significance**

In statistical hypothesis testing, is is always a possibility that some observed correlation occurred by pure chance. In that case, the correlation coefficient obtained from one of the above described methods might hold false information.

To model this phenomenon, we define a `Null-Hypothesis` $H_0$, indicating that there does not exist a correlation. Similarly, we define the `Alternative-Hypothesis` $H_A$, which is the opposite of $H_0$.

Now, we also always compute the so-called `p-value`, which is the probability of rejecting the null hypothesis $H_0$. The p-value therefore gives evidence against a null hypothesis. The smaller the p-value, the stronger the evidence that you should reject the null hypothesis.

Obviously, we have to choose some limit of the `p-value`, at which we define the correlation to be statistically significant. We therefore choose a certain significance threshold - a value that determines, whether or not we acknowledge the correlation to be significant or nonsignificant. The common choice of such a significance thresshold, denoted as $\alpha$, is `0.05`. However, it can be lowered to be increase the certainty of not accepting false positives.

When running simulatanous tests, we need to adjust the significance threshold - this is called `Bonferri Correction`. See more on this later.

## Preparing Data for Association
---
In order to associate the different variables to each other, we need to merge the two dataframes, such that for each reported day in each of Germany's regions, we have the `corona` data and the `weather` data. 

It only makes sense to include such days into our analysis that have records both for the `corona` cases and the `weather`, otherwise we cannot make an associations for this day. The merged dataframe that we will store in `DATA_GERMANY['merged']` will therefore hold less records than both other dataframes.

In order to merge the dataframes, we use Pandas `.merge()` method on dataframes. We want to merge both by `date` and the `region_code` in order to merge each day of measurement in each region, where both `corona` and `weather` data are available. We specify these parameters in the `.merge()` method.

In [ ]:
# merge dataframes both by date and region
DATA_GERMANY['merged'] = DATA_GERMANY['corona']['all'].merge(DATA_GERMANY['weather']['all'], 
                            left_on=['date', 'region_code'], right_on=['date', 'region_code'])

# minor changes in column naming for consistency
DATA_GERMANY['merged'] = DATA_GERMANY['merged'].drop(['region_y'], axis=1).rename(columns={'region_x': 'region'})

### Inspection of Merged Dataframe

In [ ]:
for dataset in NAMES['datasets']:
    print(f"{dataset.capitalize()}: {DATA_GERMANY[dataset]['all'].shape}")
print(f"Merged: {DATA_GERMANY['merged'].shape}")

We can see that we have lost some data records through the merge. This is logical, since we only want to include days into our correlation for which we have data for both our variables. 
There are two possible scenarios:
> (a) **No COVID-Data, but Weather Data** (This happens, because the `corona` data wasn't recorded on a daily basis in every region throughout the whole year)

> (b) **No Weather Data, but COVID-Data** (This happens, because our `weather` data starts on the 13th of February, whereas the first record of Covid was in January)

In [ ]:
# a. missing covid data
print(f"Days of Missing Weather Data: {DATA_GERMANY['weather']['all'].shape[0] - DATA_GERMANY['merged'].shape[0]}")
# b missing weather data
print(f"Days of Missing Covid Data: {DATA_GERMANY['corona']['all'].shape[0] - DATA_GERMANY['merged'].shape[0]}")

## Associations
---
The goal of this project is to find associations between the spread of the pandemic and several environmental conditions that have been obtained from the IBM Weather data in the same time period as the `corona` data.

For the scope of this study we chose to put particular focus on investigating the correlationn between the spread of the disease and the `UV-Index`. We chose to measure the spread of the disease through the `Absolute Number of Infection` per day per region, rather than the relative values.

As seen in the theoretical explanation of statistical testing, we need to choose a certain significance threshold, that will determine whether or not the statistical association (measured through the correlation coefficient) is significant. 

We chose a high evidential threshold of a `p-value` of 0.001. Since results from this data analysis could have major influence on the future of the country, we want to be maximally certain that the correlations we might observe are significant.

Since we are running a lot of tests, we need to correct for multiple hypothesis testing. This is the `Bonferroni correction`. If we run N tests and we determine our significance threshold as a p-value of 0.001, then we need to make sure that the p-values we look at are lower than 0.001 / N. We are multiplying the length of the list of variables that we want to associate to by two, because we are running two correlations per variable (Pearson and Spearman Correlations)

In [ ]:
# this list contains the names of the variables we want to use as predictors
associate_to = ['relative_humidity_surface', 'solar_radiation', 'surface_pressure', 'temperature_above_ground', 'total_precipiation', 'uv_index', 'wind_speed']

# choice of significance threshold (high evidential threshold)
significance_threshold = 0.001 / (len(associate_to) * 2)
print(f"Significance Threshold: {significance_threshold:.9f}")

In [ ]:
for var in associate_to:
    corr, pvalue = pearsonr(DATA_GERMANY['merged']["absolute_infections"], DATA_GERMANY['merged'][var])
    print(f"{var.title().replace('_', ' ')}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold}\n")

# alternative using pd.DataFrame().corr()
#DATA_GERMANY['merged'].corr(method='pearson').loc['absolute_infections'][associate_to].reset_index().rename(columns={'index': 'predicator', 'absolute_infections': 'correlation coefficient'})

In [ ]:
# spearman correlation
for var in associate_to:
    corr, pvalue = spearmanr(DATA_GERMANY['merged']["absolute_infections"], DATA_GERMANY['merged'][var])
    print(f"{var.title().replace('_', ' ')}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold}\n")

### Visualisation
---

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=len(associate_to), figsize=(8*len(associate_to),6))
for i, var in enumerate(associate_to):
    ax[i].scatter(DATA_GERMANY['merged'][var], DATA_GERMANY['merged']['absolute_infections']);
    ax[i].set_title(f"Scatter of {var.title().replace('_', ' ')} and No. of Infections", fontweight='bold'); 
    ax[i].set_xlabel(f"{var.title().replace('_', ' ')}"); 
    m, b = np.polyfit(DATA_GERMANY['merged'][var], DATA_GERMANY['merged']['absolute_infections'], 1)
    ax[i].plot(DATA_GERMANY['merged'][var], m*DATA_GERMANY['merged'][var] + b, 'r');
ax[0].set_ylabel('No. of Cases (per Day)');
save_figure(fig, path=PATH['reports'] + 'associations/', filename='all_associations', save_to='pdf')

In [ ]:
# alternative way of visualisation using seaborns 'pairplot'
df = DATA_GERMANY['merged'][['absolute_infections'] + associate_to]

sns.pairplot(
    df,
    x_vars = list(df),
    y_vars = ['absolute_infections', 'uv_index']) # show association betwee `absolute infections` and `uv_index` to all other environmental variables
plt.show()

## Report: Linear Correlation between `Number of Infections` and `UV-Index`
---
Although we have chosen a particulary low significance threshold, we could observe associations between the absolute number of infections per day and the weather conditions. In this section we will analyse, which conclusions can be drawn from the results for the correlation between the `Number of Infections` and the `UV-Index`

**Results** 

All three statistical tests report a *negative* association (*Average Correlation Coefficient*: ) between the absolute number of infections and the UV-Index near the surface. Namely, the higher the UV-Index, the less infections could be observed.

## Further Investigation
---
Our results from a simple association between the absolute number of new infections in different regions on each day and several weather conditions in the same regions, however have to be viewed critically. Although, we can observe statistical associations, this does not axiomatically imply a biological cause in the sense that ie. higher temperatures allow the virus to survive longer on surfaces and therefore spread faster. 

There are several other possibilities, which we call *spurious correlations*:

1. **Common-Causal Variable/ Confounder  ** (A and B are both caused by C)

2. **Mediator** (A causes C and C causes B)

3. **Coincidence** (There is no connection between A and B; the correlatin appeared by pure chance)

Since we have obtained our results from correlating roughly 5500 instances of the related variables and obtained significantly low p-values, it is highly unlikely that the observed correlations happened by pure chance. While we can rule out coincidence as a source of a spurious correlation, we can't do the same with the other two. In contrast - it seems reasonable that ie. a mediator is causing the correlation. This is, because weather influences human behavior a lot - and this changed behavior then might be the factor influencing the spread of the disease. 

In [ ]:
# We now prepare for running a multivariate linear regresion using statsmodel
# The library requires us to create a constant variable, to calculate the
# intercept.
DATA_GERMANY['merged'] = sm.add_constant(DATA_GERMANY['merged'])
associate_to.append("const")

Lots to unpack here, but let's focus on the basics. The R-squared (top-right)
is a measure of prediction quality: how much of the daily variation in number
of cases can we explain? The "P>|t|" column tells you the (non Bonferroni
corrected) p-values of each variable *when keeping all the other constant*.
For instance, this regression tells us that varying SolarRadiation doesn't
tell us anything interesting if everything else is held constant.

In [ ]:
# we run the linear multivariate regression with log-transformed number of absolute infections
mlr = sm.OLS(
    endog = np.log(DATA_GERMANY['merged']["absolute_infections"]), 
    exog = DATA_GERMANY['merged'][associate_to], 
    hasconst = True).fit()

print(mlr.summary())

## Report: Association between `UV Index` and `Absolute Infections` Attribute 
---
In `TASK 2` we wanted to find out whether there is an association between the UV Index and the absolute number of corona infections.

This 

In [ ]:
condition = 'uv_index'
fig, ax = plt.subplots(figsize=(16,9))
ax.scatter(DATA_GERMANY['merged'][condition], DATA_GERMANY['merged']['absolute_infections']);
ax.set_title('Association of UV Index and Absolute No. of Infections per Day', fontweight='bold', fontsize=14); 
ax.set_xlabel('UV Index'); ax.set_ylabel('No. of Infections per Day');
m, b = np.polyfit(DATA_GERMANY['merged'][condition], DATA_GERMANY['merged']['absolute_infections'], 1)
ax.plot(DATA_GERMANY['merged'][condition], m*DATA_GERMANY['merged'][condition] + b, 'r');

# Spatial Visualisation (TASK 3)
---
Let's now visualise the `corona` and `weather` data on an informative map. The code is are based on `folium` - an external Python package - that makes use of leaflet.js, which is a JavaScript package used to create interactive maps.
After export, the maps can be explored as an `html` file in the browser or inline within Jupyter. 

Our goal is to visualise the regional differences in both the `corona` and `weather` data on a so called `choropleth` map. A choropleth map is a type of thematic map in which a set of pre-defined areas is colored or patterned in proportion to a statistical variable that represents an aggregate summary of a geographic characteristic within each area. 

We will firstly create a static choropleth map that represents the annual aggregates for each region and secondly an interactive choropleth with a time slider in order to interactively explore the development of covid cases and environmental conditions.

## Preparing Data
---
In order to plot our data onto a choropleth map, we need to prepare it, since the `folium.Choropleth` method requires the data to be in a specific format. We will furthermore create an overlay, that enables the client to hover over each region to learn more about it. In order to be able to do this, we need to read in our `geodata` using `geopandas`. In the following process, we aggregate the `corona` (sum) and `weather` (mean) data, make some adjustments in the column representation and then merge it with the geodata obtained. 

In [ ]:
# read in geodata into geopandas dataframe (necessary for folum.features.GeoJson -> overlay function)
de_geodata = gpd.read_file(PATH['data']['raw'] + PATH['shapefiles'] + 'de.geojson')
de_geodata = de_geodata.rename(columns={'iso_3166_2': 'region_code'})

In [ ]:
# aggregate annual corona data for each region (using the sum of cases)
annual_corona_per_region = pd.DataFrame(DATA_GERMANY['corona']['all'].groupby(['region_code'])[['absolute_infections', 'absolute_deaths', 'infections_(per_100.000)', 'deaths_(per_100.000)', 'region']].sum()).reset_index()
annual_corona_per_region['region'] = data_corona_region['region_code'].map(REGION_LOOKUP['iso'])

# aggregate annual environmental data for each region (using the mean of observations)
annual_weather_per_region = pd.DataFrame(DATA_GERMANY['weather']['all'].groupby(['region_code'])[['relative_humidity_surface', 'solar_radiation', 'surface_pressure', 'temperature_above_ground', 'total_precipiation', 'uv_index', 'wind_speed']].mean()).reset_index()
annual_weather_per_region['region'] = data_weather_region['region_code'].map(REGION_LOOKUP['iso'])

In [ ]:
# merging of geodata and annual aggregated corona and weather data
annual_corona_per_region = de_geodata.merge(annual_corona_per_region, on='region_code')
annual_weather_per_region = de_geodata.merge(annual_weather_per_region, on='region_code')

In [ ]:
# add population column to datasets (in order to be included in hover over regions)
annual_corona_per_region['population'] = annual_corona_per_region['region'].map(population_map)
annual_weather_per_region['population'] = annual_weather_per_region['region'].map(population_map)

# rearranging of columns for nicer labels 
annual_corona_per_region = annual_corona_per_region[['region', 'region_code', 'population', 'absolute_infections', 'infections_(per_100.000)', 'absolute_deaths', 'deaths_(per_100.000)', 'geometry']]
annual_weather_per_region = annual_weather_per_region[['region', 'region_code', 'population', 'relative_humidity_surface', 'solar_radiation', 'surface_pressure', 'temperature_above_ground', 'total_precipiation', 'uv_index', 'wind_speed', 'geometry']]

In [ ]:
def choropleth(data, geodata, condition):
    #create the folium.Choropleth
    m = folium.Map(location = [51.3, 10.3], zoom_start = 6, tiles='cartodbpositron')

    # plot onto chloropleth
    folium.Choropleth(
        geo_data = geodata,
        data = data,
        columns = ["region_code", condition],
        key_on = "properties.iso_3166_2",
        fill_color = "OrRd",
        fill_opacity = .6,
        line_opacity = .5,
        legend_name = f"{condition.title().replace('_', ' ')} (Time Period: February 2020 - February 2021)",
    ).add_to(m)

    style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color':'#000000', 
                                    'fillOpacity': 0.50, 
                                    'weight': 0.1}
    
    attributes = list(data)
    fields = attributes[:-1]
    aliases = [attribute.title().replace('_',' ') for attribute in fields]

    overlay = folium.features.GeoJson(
        data,
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(
            fields = fields,
            aliases = aliases,
            style = ("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )
    m.add_child(overlay)
    m.keep_in_front(overlay)
    folium.LayerControl().add_to(m)

    return m

## Report: Relevant Map
---
Since we are investigating the association between the `Absolute Number of Cases` and the `UV Index`, we are trying to get a visual intuition for these two variables on our choropleth map. We plot them in the following section.

### UV-Index

In [ ]:
annual_uv_indexes = choropleth(
    data=annual_weather_per_region, 
    geodata = DATA_GERMANY['shapefiles'], 
    condition='uv_index')
annual_uv_indexes

### Relative Annual Infections
---

In [ ]:
annual_relative_infections = choropleth(
    data=annual_corona_per_region, 
    geodata = DATA_GERMANY['shapefiles'], 
    condition='infections_(per_100.000)')
annual_relative_infections

### Save Maps
---
Let's save all maps for later inspection using our `save_map` function. All maps will be saved into `../reports/maps/corona/annual` or `../reports/maps/weather/annual`

In [ ]:
for condition in ['absolute_infections', 'infections_(per_100.000)', 'absolute_deaths', 'deaths_(per_100.000)']:
    save_map(choropleth(
        data = annual_corona_per_region,
        geodata = DATA_GERMANY['shapefiles'],
        condition = condition
    ), path=PATH['reports'] + 'maps/corona/annual/' , filename=f'annual_{condition}')

In [ ]:
for condition in ['relative_humidity_surface', 'solar_radiation', 'surface_pressure', 'temperature_above_ground', 'total_precipiation', 'uv_index', 'wind_speed']:
    save_map(choropleth(
        data = annual_weather_per_region,
        geodata = DATA_GERMANY['shapefiles'],
        condition = condition
    ), path=PATH['reports'] + 'maps/weather/annual/' , filename=f'annual_{condition}')

## Digress: Interactive Time Slider Choropleth
---
This section contains code for plotting an interactive choropleth map that shows the number of infections aggregated by either Week or Month through an interactive time slider. This part was not mandatory for TASK 3, but was researched on own initiative in the hope of making an even more meaningful visualisation of the given data.
The code depends on the `TimeSliderChoropleth` object within `folium.plugins` that is not well-maintained, but serves the purpose for this visualisation.

In [ ]:
from folium.plugins import TimeSliderChoropleth
import branca.colormap as cm

In [ ]:
def interactive_choropleth(data, condition, aggregation, aggregated_by='sum'):
    if aggregated_by == 'sum':
        if aggregation == 'daily':
            to_plot = pd.DataFrame(data.groupby(['region_code', 'date'])[condition].sum()).reset_index()
        elif aggregation == 'weekly': 
            to_plot = pd.DataFrame(data.groupby(['region_code', pd.Grouper(key='date', freq='W-Mon')])[condition].sum()).reset_index()
        elif aggregation == 'monthly': 
            to_plot = pd.DataFrame(data.groupby(['region_code', pd.Grouper(key='date', freq='M')])[condition].sum()).reset_index()
    elif aggregated_by == 'mean':
        if aggregation == 'daily':
            to_plot = pd.DataFrame(data.groupby(['region_code', 'date'])[condition].mean()).reset_index()
        elif aggregation == 'weekly': 
            to_plot = pd.DataFrame(data.groupby(['region_code', pd.Grouper(key='date', freq='W-Mon')])[condition].mean()).reset_index()
        elif aggregation == 'monthly': 
            to_plot = pd.DataFrame(data.groupby(['region_code', pd.Grouper(key='date', freq='M')])[condition].mean()).reset_index()

    to_plot = to_plot.merge(de_geodata, on='region_code')
    to_plot['date_sec'] = pd.to_datetime(to_plot['date']).astype(int) / 10**9
    to_plot['date_sec'] = to_plot['date_sec'].astype(int).astype(str)

    # coloring (using branca.colormap)
    max_colour = max(to_plot[condition])
    min_colour = min(to_plot[condition])
    cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
    to_plot['colour'] = to_plot[condition].map(cmap)

    # building style dict (taken from external resource)
    region_code_list = to_plot['region_code'].unique().tolist()
    style_dict = {}
    for i in range(len(region_code_list)):
        region = region_code_list[i]
        result = to_plot[to_plot['region_code'] == region]
        inner_dict = {}
        for _, r in result.iterrows():
            inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
        style_dict[str(i)] = inner_dict

    # building country data dict
    countries_df = to_plot[['geometry']]
    countries_gdf = gpd.GeoDataFrame(countries_df)
    countries_gdf = countries_gdf.drop_duplicates().reset_index()

    # plot on folium map using `TimeSliderChoropleth`
    slider_map = folium.Map(location = [51.3, 10.3], zoom_start = 6, tiles='cartodbpositron')

    _ = TimeSliderChoropleth(
        data=countries_gdf.to_json(),
        styledict=style_dict,
    ).add_to(slider_map)

    _ = cmap.add_to(slider_map)
    cmap.caption = f"{condition.title().replace('_', ' ')} ({aggregation.title()})"

    return slider_map

### Report: Relevant Maps
--- 
Again, we are trying to get a visual intuition for our two focused variables `Infections` and `UV Index` on our choropleth map. We plot them in the following section.

In [ ]:
weekly_relative_infections = interactive_choropleth(
    data = DATA_GERMANY['corona']['all'], 
    condition = 'infections_(per_100.000)', 
    aggregation='weekly', aggregated_by='sum')

weekly_relative_infections

In [ ]:
weekly_uv_index = interactive_choropleth(
    data = DATA_GERMANY['weather']['all'], 
    condition = 'uv_index', 
    aggregation='weekly', aggregated_by='mean')

weekly_uv_index

### Save Maps
---

In [ ]:
for aggregation in ['daily', 'weekly', 'monthly']:
    for condition in ['absolute_infections', 'infections_(per_100.000)', 'absolute_deaths', 'deaths_(per_100.000)']:
        save_map(interactive_choropleth(
            data = DATA_GERMANY['corona']['all'],
            condition = condition,
            aggregation = aggregation, aggregated_by='mean'), path=PATH['reports'] + 'maps/corona/' + f'{aggregation}', filename=f"{aggregation}_{condition}")

In [ ]:
for aggregation in ['daily', 'weekly', 'monthly']:
    for condition in ['relative_humidity_surface', 'solar_radiation', 'surface_pressure', 'temperature_above_ground', 'total_precipiation', 'uv_index', 'wind_speed']:
        save_map(interactive_choropleth(
            data = DATA_GERMANY['weather']['all'],
            condition = condition,
            aggregation = aggregation, aggregated_by='mean'), path=PATH['reports'] + 'maps/weather/' + f'{aggregation}', filename=f"{aggregation}_{condition}")

# Further Associations (TASK 4)
---
`TASK 2` has proven some correlations between environmental conditions and the spread of the disease. We, however, also concluded that it would be fatal to derive a *causation* chain here. That some change in one variable leads to a change in another variable does not mean at all, that there exists a logical causation relation between the two variables. 

In the example of the UV-Index, that is negatively correlated to the absolute number of infections, there might be other *mediator* or *confounder* variables that may explain this correlation. In section 4 we are trying to investigate more variables an their influence on the spread of the disease with the goal of trying to make more sense of which factors actually rule the spread of the pandemic.

## 1. Gathering External Variables
---
To do so, we must find more variables that we can relate to the daily number of infections for each region and that seem logical to associate, ie. it is unreasonable to associate the averaged soil pH-value per day per region to the number of infections, because we can say with very high confidence that there does not exist any correlation.

### Lockdown
The obvious choice of parameter is of course the periods of lockdown. It is important for the leaders to know, whether or not the lockdown was an efficient method of combating the spread of the disease. Or statistically: Does an increase in lockdown, decrease the number of infections? (Negative Correlation). 

In order to check for this parameter, we have obtained the dates for the two periods of lockdown employed by the german authorities [Wikipedia](https://en.wikipedia.org/wiki/COVID-19_pandemic_in_Germany). Lockdown in this context is understood as a complete closing of all shops, social distancing and mandatory mask in all public places. The variable was implemented as a binary value with `0` referring to no lockdown and `1` referring to lockdown as described above.

1. Lockdown: 22.03.2020 - 15.06.2020

2. Lockdown: 16.12.2020 - `today`

In [ ]:
lockdown1_mask = (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 3, 22)) & (DATA_GERMANY['merged']['date'] <= datetime.datetime(2020, 6, 15))
lockdown2_mask = (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 12, 16))
lockdown_mask = lockdown1_mask | lockdown2_mask
DATA_GERMANY['merged']['lockdown'] = lockdown_mask.astype(int)

### Holiday
Similarly to the periods of lockdown, we also want to research, whether or not there the periods of holiday had an influence on the spread of the pandemic, ie. the number of cases might have peaked, since people travel more and have more social contacts, or the number of cases might have dropped, since every-day contacts at school and work are reduced.

We obtained the time periods and masked our data with it, in order to create a binary variable, where `0` indicates no holiday and `1` indicates a holiday period.

In [ ]:
# referenced from most populated region - bavaria (since holiday times differ from region to region) (https://www.schulferien.org/deutschland/ferien/2020/)
easter_holiday =  (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 4, 6)) & (DATA_GERMANY['merged']['date'] <= datetime.datetime(2020, 4, 18))
pfingst_holiday = (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 6, 2)) & (DATA_GERMANY['merged']['date'] <= datetime.datetime(2020, 6, 13))
summer_holiday =  (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 7, 30)) & (DATA_GERMANY['merged']['date'] <= datetime.datetime(2020, 9, 12))
fall_holiday =  (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 10, 30)) & (DATA_GERMANY['merged']['date'] <= datetime.datetime(2020, 11, 6))
christmas_holiday =  (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 12, 21)) & (DATA_GERMANY['merged']['date'] <= datetime.datetime(2021, 1, 9))
winter_break =  (DATA_GERMANY['merged']['date'] >= datetime.datetime(2021, 2, 24)) & (DATA_GERMANY['merged']['date'] <= datetime.datetime(2021, 2, 28))

# https://www.ferienwiki.de/feiertage/2020/de
special_days =  (DATA_GERMANY['merged']['date'] == datetime.datetime(2021, 1, 6)) & (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 2, 15)) & (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 5, 1)) & (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 5, 8)) & (DATA_GERMANY['merged']['date'] >= datetime.datetime(2020, 5, 21)) & (DATA_GERMANY['merged']['date'] <= datetime.datetime(2020, 10, 3))  

holiday = easter_holiday | pfingst_holiday | summer_holiday | fall_holiday | christmas_holiday | winter_break | special_days
DATA_GERMANY['merged']['holiday'] = holiday.astype(int)

### Area and Population Density

Lastly, we want to research, whether or not the area and the population density of the region we are looking at has influenced the disease spread. We would expect that the absolute number of cases is positively correlated to the area of the region, since the more people there are in the region, the more infections we can expect statistically. Likewise, the population density should also be positively correlated, since the denser the region, the more social contact are expected and therefore more relative infections. 

To implement this variable, we obtained the areas in $km^2$ for each region (*[Wikipedia](https://en.wikipedia.org/wiki/States_of_Germany)*) and mapped it onto our dataframe into a new column. The population density could then be easily computed by dividing the population size of each region (which is already part of the dataframe) by the area.

In [ ]:
# area dictionary holding the area in km2 for each region (key: iso3166-2)
area_dict = {"DE-BW": 35752, "DE-BY": 70552, "DE-BE": 892, "DE-BB": 29479, "DE-HB": 419, "DE-HH": 755, "DE-HE": 21115, "DE-NI": 47609, "DE-MV": 23180, "DE-NW": 34085, "DE-RP": 19853, "DE-SL": 2569, "DE-SN": 18416, "DE-ST": 20446, "DE-SH": 15799, "DE-TH": 16172}

DATA_GERMANY['merged']['area'] = DATA_GERMANY['merged']['region_code'].map(area_dict)
DATA_GERMANY['merged']['population_density'] = DATA_GERMANY['merged']['population'] / DATA_GERMANY['merged']['area']

## 2. Associations
---
We now want to repeat our associations - first, one-by-one using both *Pearson* and *Spearman* Correlations and secondly, through a multiple linear regression.

### Pearson/ Spearman Correlations

In [ ]:
new_associate_to = ['lockdown', 'holiday', 'area', 'population_density']
significance_threshold = (0.001 / len(a)) * 2

In [ ]:
for var in new_associate_to:
    corr, pvalue = pearsonr(DATA_GERMANY['merged']["absolute_infections"], DATA_GERMANY['merged'][var])
    print(f"{var.title().replace('_', ' ')}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold}\n")

In [ ]:
for var in new_associate_to:
    corr, pvalue = spearmanr(DATA_GERMANY['merged']["absolute_infections"], DATA_GERMANY['merged'][var])
    print(f"{var.title().replace('_', ' ')}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold}\n")

### Multiple Linear Regression

In [ ]:
associate_to = associate_to + ['lockdown', 'holiday', 'area', 'population_density']


In [ ]:
DATA_GERMANY['merged'] = sm.add_constant(DATA_GERMANY['merged'])
associate_to.append("const")

# we run the linear multivariate regression with log-transformed number of absolute infections
mlr = sm.OLS(
    endog = np.log(DATA_GERMANY['merged']["absolute_infections"]), 
    exog = DATA_GERMANY['merged'][associate_to], 
    hasconst = True).fit()

print(mlr.summary())

## 3. Summary
---
